#Colab을 이용한 Kaggle 리더보드 파일 제출

In [1]:
!pip install kaggle #Kaggle 패키지 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from google.colab import files
#files.upload()

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c prml23-1

  0% 0.00/1.59k [00:00<?, ?B/s]
100% 1.59k/1.59k [00:00<00:00, 722kB/s]


In [4]:
!ls

kaggle.json  prml23-1.zip  sample_data


In [6]:
!unzip prml23-1.zip

Archive:  prml23-1.zip
  inflating: iris-test.csv           
  inflating: iris-train.csv          
  inflating: sample_submit.csv       


In [ ]:
import pandas as pd

#데이터
xy_data = pd.read_csv('iris-train.csv')
X_test = pd.read_csv('iris-test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

X = xy_data.drop('species', axis=1) #'species'열을 drop하고 input X를 정의함
classle=LabelEncoder() #class label
#'species'열의 문자열을 categorical value로 전환
y=classle.fit_transform(xy_data['species'].values) 
X.shape

(105, 4)

In [ ]:
from sklearn.model_selection import train_test_split

#X(입력변수)와 y(종속변수)로 이루어진 데이터를 학습과 테스트 데이터넷으로 각각 80%, 20%의 비율로 나눔
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size=0.2, random_state=1, stratify=y) 
print('입력변수(학습데이터)    : ',  X_train.shape)
print('입력변수(검증데이터)  : ', X_valid.shape)
print('종속변수(학습데이터)    : ',  y_train.shape)
print('종속변수(검증데이터)  : ', y_valid.shape)

입력변수(학습데이터)    :  (84, 4)
입력변수(검증데이터)  :  (21, 4)
종속변수(학습데이터)    :  (84,)
종속변수(검증데이터)  :  (21,)


In [ ]:
#KNN의 적용
from sklearn.neighbors import KNeighborsClassifier  #KNN 불러오기
knn = KNeighborsClassifier(n_neighbors=5, p=2) #5개의 인접한이웃, 거리측정기준:유클리드
knn.fit(X_train,y_train) #모델 fitting과정

KNeighborsClassifier()

In [ ]:
y_train_pred = knn.predict(X_train) #train data의 y값 예측치
y_valid_pred  = knn.predict(X_valid)  #모델을 적용한 test data의 y값 예측치
print('Misclassified training samples: %d' %(y_train!=y_train_pred).sum()) #학습데이터로 오분류 갯수 확인
print('Misclassified test samples: %d' %(y_valid!=y_valid_pred).sum()) #테스트데이터로 오분류 갯수 확인

Misclassified training samples: 1
Misclassified test samples: 1


In [ ]:
from sklearn.metrics import accuracy_score #정확도 계산을 위한 모듈 import
print(accuracy_score(y_valid,y_valid_pred))

0.9523809523809523


In [ ]:
from sklearn.metrics import confusion_matrix 
#대각원소가 각각 setosa, versicolor, virginica를 정확하게 분류한 갯수
conf=confusion_matrix(y_true=y_valid,y_pred=y_valid_pred) 
print(conf) #confusion matrix 출력

[[7 0 0]
 [0 6 1]
 [0 0 7]]


In [ ]:
y_test_pred  = knn.predict(X_test)  #모델을 적용한 test data의 y값 예측치
y_test_pred_org = classle.inverse_transform(y_test_pred) #원래의 species 문자열로 복원

In [ ]:
submit = pd.read_csv('sample_submit.csv')
submit['species']= y_test_pred_org
submit.to_csv('my-answer3.csv', mode='w', header= True, index= False) #제출파일 생성